In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.sparse as sp
import re
import random

import torch

In [ ]:
graph_data = pd.read_csv('./data/baci_pre_graph.csv')
node_data = pd.read_csv('./data/baci_node_features.csv')

In [ ]:
graph_data

In [ ]:
node_data

In [ ]:
not_in_lst = []
for n in range(len(graph_data)):
    if graph_data.loc[n]['Exporter'] not in node_data['node'].unique() or graph_data.loc[n]['Importer'] not in node_data['node'].unique():
        not_in_lst.append(graph_data.index.values[n])

graph_data.drop(not_in_lst, axis=0, inplace=True)

In [ ]:
graph_data

In [ ]:
features = np.array(graph_data.iloc[:, 6:], dtype=np.float32)
label = np.array(graph_data.iloc[:, 5], dtype=np.float32)
edges_unordered = np.array(graph_data[['Exporter', 'Importer']], dtype=np.int32)
idx = np.array(node_data.iloc[:, 0], dtype=np.int32)
idx_map = {p:n for n, p in enumerate(idx)}
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(graph_data['GDP'].shape[0], graph_data['GDP'].shape[0]), dtype=np.float32).toarray()
edge_attr = np.array(graph_data['Value(usd)'], dtype=np.float32)

In [ ]:
features.shape, label.shape

In [ ]:
edge_index = torch.tensor(edges, dtype=torch.int64).t().contiguous()

In [ ]:
edge_index.shape

In [ ]:
edge_feat = torch.tensor(edge_attr, dtype=torch.float32).unsqueeze(1)

In [ ]:
edge_feat.shape

In [ ]:
train_index = int(0.6 * features.shape[0])
val_index = int(0.6 * features.shape[0]) + int(0.2 * features.shape[0])

X_train = features[:train_index]
X_val = features[train_index:val_index]
X_test = features[val_index:]
y_train = label[:train_index]
y_val = label[train_index:val_index]
y_test = label[val_index:]

In [ ]:
X_train.shape, X_test.shape, X_val.shape

In [ ]:
y_train.shape, y_test.shape, y_val.shape

In [ ]:
np.savetxt('./data/graph_idx.csv', features, delimiter=',')
np.savetxt('./data/graph_label.csv', label, delimiter=',')

torch.save(edge_index, './data/edge_index.pt')
torch.save(edge_feat, './data/edge_attr.pt')

np.savetxt('./data/X_train.csv', X_train, delimiter=',')
np.savetxt('./data/X_val.csv', X_val, delimiter=',')
np.savetxt('./data/X_test.csv', X_test, delimiter=',')
np.savetxt('./data/y_train.csv', y_train, delimiter=',')
np.savetxt('./data/y_val.csv', y_val, delimiter=',')
np.savetxt('./data/y_test.csv', y_test, delimiter=',')